In [23]:

!pip install neo4j langchain-community langchain_neo4j langchain_groq

from langchain_neo4j import Neo4jGraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 4.4 MB/s eta 0:00:00


In [11]:
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph

graph = Neo4jGraph(url="neo4j+s://2b807bcc.databases.neo4j.io", username="neo4j", password="O-pjgIe8i-HUDXWu13AB1PGycfuUm5wmxTO1kkYgywk")

In [17]:
movie_query = """
    LOAD CSV WITH HEADERS FROM
    'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
    AS row

    MERGE (m:Movie{id:row.movieId})

    SET m.released = date(row.released),
        m.title = row.title,
        m.imdbRating = toFloat(row.imdbRating)

    FOREACH (director IN split(row.director, '|') |
      MERGE (p:Person{name:trim(director)})
      MERGE (p)-[:DIRECTED]->(m)
    )

    FOREACH (actor IN split(row.actors, '|') |
      MERGE (p:Person{name:trim(actor)})
      MERGE (p)-[:ACTED_IN]->(m)
    )

    FOREACH (genre IN split(row.genres, '|') |
      MERGE (g:Genre{name:trim(genre)})
      MERGE (m)-[:IN_GENRE]->(g)
    )
    """

In [18]:
graph.query(movie_query)

[]

In [19]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [26]:
import os

groq_api_key="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"
print(groq_api_key)

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


In [27]:
from langchain_groq import ChatGroq
llm=ChatGroq(api_key=groq_api_key, model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a4f4e5007d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a4f4e4734d0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [30]:
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
chain=GraphCypherQAChain.from_llm(llm, graph=graph, verbose=True ,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_neo4j.graphs.neo4j_graph.Neo4jGraph object at 0x7a4f4f383190>, cypher_generation_chain=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}')
| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a4f4e5007d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a4f4e4734d0>, model_name='Gemma2-9b-It', model_kwargs

In [31]:
response=chain.invoke({"query":"Who was the director of the movie Casino?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})<-[:DIRECTED]-(p:Person) RETURN p.name
Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director of the movie Casino?',
 'result': 'Martin Scorsese  was the director of the movie Casino. \n'}

In [32]:
response=chain.invoke({"query":"Who was the actors of the movie Casino?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(p:Person) RETURN p.name 

Full Context:
[{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}]

> Finished chain.


{'query': 'Who was the actors of the movie Casino?',
 'result': 'Robert De Niro, Joe Pesci, Sharon Stone, and James Woods.  \n'}

In [33]:
response=chain.invoke({"query":"Display the actors who acted in multiple movies?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WITH p, count(m) AS numMovies
WHERE numMovies > 1
RETURN p.name
Full Context:
[{'p.name': 'Tim Allen'}, {'p.name': 'Tom Hanks'}, {'p.name': 'Robin Williams'}, {'p.name': 'Walter Matthau'}, {'p.name': 'Sophia Loren'}, {'p.name': 'Angela Bassett'}, {'p.name': 'Steve Martin'}, {'p.name': 'Al Pacino'}, {'p.name': 'Robert De Niro'}, {'p.name': 'Val Kilmer'}]

> Finished chain.


{'query': 'Display the actors who acted in multiple movies?',
 'result': "I don't know the answer. \n"}